BDLE 2021

date du document  :  

# TP accès parallèle : EAU 

exemple d'accès en parallèle aux données du site Eau de France.
[Données de piezométrie](https://hubeau.eaufrance.fr/page/api-piezometrie) 

## Préparation

Pour accéder directement aux fichiers stockées sur votre google drive. Renseigner le code d'authentification lorsqu'il est demandé

Ajuster le nom de votre dossier : MyDrive/essai

In [ ]:
# import os
# from google.colab import drive
# drive.mount("/content/drive")

# drive_dir = "/content/drive/MyDrive/essai"
# os.makedirs(drive_dir, exist_ok=True)
# os.listdir(drive_dir)

Installer pyspark et findspark :


In [ ]:
!pip install -q pyspark
!pip install -q findspark
print("installé")

     |████████████████████████████████| 281.3 MB 31 kB/s 
     |████████████████████████████████| 198 kB 50.1 MB/s 
installé


Démarrer la session spark

In [ ]:
import os
# !find /usr/local -name "pyspark"
os.environ["SPARK_HOME"] = "/usr/local/lib/python3.7/dist-packages/pyspark"
os.environ["JAVA_HOME"] = "/usr"

In [ ]:
# Principaux import
import findspark
from pyspark.sql import SparkSession 
from pyspark import SparkConf  

# pour les dataframe et udf
from pyspark.sql import *  
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *

# pour le chronomètre
import time

# initialise les variables d'environnement pour spark
findspark.init()

# Démarrage session spark 
# --------------------------
def demarrer_spark():
  local = "local[*]"
  appName = "TP"
  configLocale = SparkConf().setAppName(appName).setMaster(local).\
  set("spark.executor.memory", "6G").\
  set("spark.driver.memory","3G").\
  set("spark.sql.catalogImplementation","in-memory")
  
  spark = SparkSession.builder.config(conf = configLocale).getOrCreate()
  sc = spark.sparkContext
  sc.setLogLevel("ERROR")
  
  spark.conf.set("spark.sql.autoBroadcastJoinThreshold","-1")

  # On ajuste l'environnement d'exécution des requêtes à la taille du cluster (4 coeurs)
  spark.conf.set("spark.sql.shuffle.partitions","4")    
  print("session démarrée, son id est ", sc.applicationId)
  return spark
spark = demarrer_spark()

session démarrée, son id est  local-1640477000931


Redéfinir la fonction **display** pour afficher le resutltat des requêtes dans un tableau

In [ ]:
import pandas as pd
from google.colab import data_table

# alternatives to Databricks display function.

def display(df, n=100):
  return data_table.DataTable(df.limit(n).toPandas(), include_index=False, num_rows_per_page=10)

def display2(df, n=20):
  pd.set_option('max_columns', None)
  pd.set_option('max_colwidth', None)
  return df.limit(n).toPandas().head(n)

Définir le tag **%%sql** pour pouvoir écrire plus simplement des requêtes en SQL dans une cellule

In [ ]:
from IPython.core.magic import (register_line_magic, register_cell_magic, register_line_cell_magic)

def removeComments(query):
  result = ""
  for line in query.split('\n'):
    if not(line.strip().startswith("--")):
      result += line + " "
  return result

@register_line_cell_magic
def sql(line, cell=None):
    "To run a sql query. Use:  %%sql"
    val = cell if cell is not None else line
    tabRequetes = removeComments(val).split(";")
    for r in tabRequetes:
        if len(r.strip()) > 2:
          derniere = spark.sql(r)
    return display(derniere)
print("fonctions définies")

fonctions définies


In [ ]:
#------------------------------
# Chronometre : chronoPersist2
#------------------------------
import time

# Ce chronometre garantit que chaque tuple du dataframe est lu entièrement.
# En effet il est nécessaire de lire le détail de chaque tuple avant de les 'copier' en mémoire.
def chronoPersist(df):
    df.unpersist()
    t1 = time.perf_counter()
    count = df.persist().count()
    t2 = time.perf_counter()
    df.unpersist()
    print('durée: {:.1f} s'.format(t2 - t1), 'pour lire', count , 'elements')

def chronoPersist2(df):
  dest = df.selectExpr("*", "1")
  t1 = time.perf_counter()
  count = dest.persist().count()
  t2 = time.perf_counter()
  dest.unpersist()
  print('durée: {:.1f} s'.format(t2 - t1), 'pour lire', count , 'elements')
        
def chronoCount(df):
  t1 = time.perf_counter()
  count = df.count()
  t2 = time.perf_counter()
  print('durée: {:.1f} s'.format(t2 - t1), 'pour dénombrer', count , 'elements')

    
print("fonctions définies")

fonctions définies


## Accès aux données

In [ ]:
import os
temp = "/temp/"
os.makedirs(temp, exist_ok=True)
os.listdir(temp)

[]

URL pour l'accès aux datasets

In [ ]:
# ---------------------------------------------------------------------------
# en cas de problème avec le téléchargement des datasets, aller directement sur l'URL ci-dessous
PUBLIC_DATASET_URL = "https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4" 
PUBLIC_DATASET=PUBLIC_DATASET_URL + "/download?path="

print("URL pour les datasets ", PUBLIC_DATASET_URL)

URL pour les datasets  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4


In [ ]:
import os
from urllib import request

# acces online aux données
# https://www.egc.asso.fr/wp-content/uploads/data_Rostrenen.csv


def load_file(file):
  if(os.path.isfile( temp + file)):
    print(file, "is already stored")
  else:
    url = PUBLIC_DATASET + "/defi_EGC_2022/" + file
    print("downloading from URL: ", url, "save in : " + temp + file)
    request.urlretrieve(url , temp  + file)

load_file("points_eau.csv")
load_file("data_Rostrenen.csv")

# Liste des fichiers de IMDB
os.listdir(temp)

downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=/defi_EGC_2022/points_eau.csv save in : /temp/points_eau.csv
downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=/defi_EGC_2022/data_Rostrenen.csv save in : /temp/data_Rostrenen.csv


['data_Rostrenen.csv', 'points_eau.csv']

## Table Point : les points d'eau

In [ ]:
point = spark.read.option("delimiter",";").option("header",True).csv(temp + 'points_eau.csv').repartition(4).persist()
point.createOrReplaceTempView("point")
display(point, 3)

,CODE_BSS,BSS_ID,LONGITUDE,LATITUDE,CODE_INSEE_COMMUNE,NOM_COMMUNE,CODE_STATION_HYDRO,NOM_STATION_HYDRO,CODE_BDLISA,NOM_ENTITE_BDLISA
0,00762X0004/S1,BSS000FHYM,0.860993857091503,49.6497369721205,76456,MOTTEVILLE,H9923020,L'Austreberthe à Saint-Paer,121AU01,Craie du Séno-Turonien du Bassin Parisien de l...
1,07223C0113/S,BSS001URLS,4.9003527411453,45.6473838620031,69273,CORBAS,None,None,521AK00,"NV3 absent, nom de l'entité NV2 : Formations f..."
2,00487X0015/S1,BSS000EECH,3.07279822338651,49.9035922409316,80413,HANCOURT,E6351408,Haute Somme à Ham,121BB01,Craie du Séno-Turonien du bassin versant de la...


In [ ]:
point.count()

18

## Requetes d'analyse

In [ ]:
%%sql
select *
from Point
where nom_commune like '%P%'

,CODE_BSS,BSS_ID,LONGITUDE,LATITUDE,CODE_INSEE_COMMUNE,NOM_COMMUNE,CODE_STATION_HYDRO,NOM_STATION_HYDRO,CODE_BDLISA,NOM_ENTITE_BDLISA
0,07476X0029/S,BSS001VTUD,5.19017582950695,45.3650549559674,38300,PENOL,None,None,521AM00,"NV3 absent, nom de l'entité NV2 : Alluvions fl..."
1,01516X0004/S1,BSS000LETA,1.61497845782586,48.9815660459082,78484,PERDREAUVILLE,None,None,121AZ01,Craie du Séno-Turonien du Bassin Parisien du V...
2,06505X0080/FORC,BSS001REHG,4.76295450181247,46.1383028294756,69242,TAPONAS,None,None,507AD02,Sables pliocènes du Val de Saône
3,00755X0006/S1,BSS000FHCQ,0.419809838577633,49.5555270637694,76714,LES TROIS-PIERRES,G9103020,Lézarde à Montivilliers,121AU01,Craie du Séno-Turonien du Bassin Parisien de l...
4,02706X0074/S77-20,BSS000UTLD,6.9302772399532,48.4395463138954,88082,CELLES-SUR-PLAINE,None,None,143AK07,Grès d'Annweiler et Grès de Senones du Buntsan...


In [ ]:
%%sql
select min(longitude)
from Point


,min(longitude)
0,-3.30717741173568


## Accès Open data online

Example d'URL : https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/chroniques?code_bss=07548X0009/F&sort=desc&size=10


In [ ]:
import urllib.request
import json


# code_bss = "07548X0009/F"
code_bss = "01516X0004/S1"
#code_bss = "00471X0095/PZ2013"
urlEau = f"https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/chroniques?code_bss={code_bss}&sort=desc&size=20"


with urllib.request.urlopen(urlEau) as response:
   encoding = response.info().get_content_charset('utf-8')
   obj = json.loads(response.read().decode(encoding))

jrdd = spark.sparkContext.parallelize([json.dumps(obj)])
data1 = spark.read.json(jrdd)
# data1.printSchema()
data2 = data1.selectExpr("explode(data) as item").\
selectExpr("item.code_bss as code",\
           "item.profondeur_nappe as niveau",\
           "item.date_mesure as date",\
           "item.timestamp_mesure as time")
data3 = data2.orderBy(desc(data2.date))
data3.persist()
data3.printSchema()
display(data3)

root
 |-- code: string (nullable = true)
 |-- niveau: double (nullable = true)
 |-- date: string (nullable = true)
 |-- time: long (nullable = true)



,code,niveau,date,time
0,01516X0004/S1,20.40,2021-12-15,1639540800000
1,01516X0004/S1,20.41,2021-12-14,1639440000000
2,01516X0004/S1,20.41,2021-12-13,1639353600000
3,01516X0004/S1,20.41,2021-12-12,1639270800000
4,01516X0004/S1,20.41,2021-12-11,1639180800000
5,01516X0004/S1,20.40,2021-12-10,1639094400000
6,01516X0004/S1,20.40,2021-12-09,1639090800000
7,01516X0004/S1,20.41,2021-12-08,1638921600000
8,01516X0004/S1,20.40,2021-12-07,1638896400000
9,01516X0004/S1,20.42,2021-12-06,1638806400000


# Exercice 1 : Accès en parallèle aux mesures d'eau

## Question 1 : Accès parallèle

Pour toutes les stations (code_bss) de la table Point. Accéder en parallèle au site pour récupérer les 10 dernières mesures de niveau d'eau.
Le résultat est un seul dataframe Mesure(code,niveau,date,time) contenant toutes les mesures de tous les points d'eau. Rendre persistant le dataframe Mesure.

In [ ]:
import urllib.request
import json

mesSize = 10

@udf(ArrayType(MapType(StringType(),StringType())))

def get_mesure(code_bss):

  urlEau = f"https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/chroniques?code_bss={code_bss}&sort=desc&size={mesSize}"

  with urllib.request.urlopen(urlEau) as response:
    encoding = response.info().get_content_charset('utf-8')
    obj = json.loads(response.read().decode(encoding))
    return obj['data']



In [ ]:
def create_mesure(point):
  mesure = point.select(point.CODE_BSS,get_mesure(point.CODE_BSS).alias("mesure"))
  data = mesure.select(col("CODE_BSS"),explode(col("mesure")).alias("m"))
  data2 = data.select(col("CODE_BSS").alias("code"),col("m.profondeur_nappe").alias("niveau"),col("m.date_mesure").alias("date"),col("m.timestamp_mesure").alias("time")).orderBy("date")

  return data2

mes = create_mesure(point)
mes.persist()
display(mes)

,code,niveau,date,time
0,00755X0006/S1,83.54,2021-11-17,1637107200000
1,00766X0004/S1,67.88,2021-11-17,1637107200000
2,00755X0006/S1,83.55,2021-11-18,1637193600000
3,00766X0004/S1,67.91,2021-11-18,1637193600000
4,00755X0006/S1,83.55,2021-11-19,1637280000000
...,...,...,...,...
95,07476X0029/S,28.86,2021-12-10,1639177200000
96,01584X0023/LV3,16.62,2021-12-10,1639177200000
97,01258X0020/S1,14.82,2021-12-10,1639177200000
98,04398X0002/SONDAG,23.66,2021-12-10,1639177200000


## Question 2 : degré de parallélisme

En faisant varier le nombre de partitions de 1 à 2, comparer la durée pour générer le dataframe Mesure.
Calculer le rapport des durées.

In [ ]:
point1Partition = point.repartition(1).persist()
point1Partition.count()

mes = create_mesure(point1Partition)
chronoPersist(mes)
chronoCount(mes)
  
point2Partitions = point.repartition(5).persist()
point2Partitions.count()



mes = create_mesure(point2Partitions)
chronoPersist(mes)
chronoCount(mes)


durée: 26.2 s pour lire 180 elements
durée: 14.2 s pour dénombrer 180 elements
durée: 16.4 s pour lire 180 elements
durée: 7.9 s pour dénombrer 180 elements


# Exercice 2 : Traitement avec des partitions

On fournit des fonctions pour afficher le contenu des partitions d'un dataframe


#### Fonctions showPartitions et showPartitionSize
 
* showPartitions : affiche les _n_ premiers éléments de chaque partition
* showPartitionSize : affiche le nombre d'éléments dans chaque partition

In [ ]:
# fonction auxilliaire
def partSize(partID, iterateur):
  c=0
  suivant = next(iterateur, None)
  while suivant is not None :
    c+=1
    suivant = next(iterateur, None)
  return [(partID, c)]


def showPartitionSize(df):  
  t = df.selectExpr("1").rdd.mapPartitionsWithIndex(partSize)
  for (partID, nbElt) in t.collect():
    print("partition", partID, ":", nbElt, "éléments")
  print()


def showPartitions(df, N=5):
  size = df.selectExpr("1").rdd.mapPartitionsWithIndex(partSize).collectAsMap()
  
  def topN(partID, iterateur):
    c=0
    head=[]
    suivant = next(iterateur, None)
    while suivant is not None and c < N :
      c+=1
      head.append(suivant)
      suivant = next(iterateur, None)
    return [(partID, head)]  
  t = df.rdd.mapPartitionsWithIndex(topN)
  for (partID, head) in t.collect():
    print("Partition", partID, ",", size[partID], "éléments")
    for row in head:
        print(row)
    print()
    
print('showPartitions et showPartitionSize définies')

showPartitions et showPartitionSize définies


In [ ]:
showPartitionSize(data3)

partition 0 : 5 éléments
partition 1 : 5 éléments
partition 2 : 5 éléments
partition 3 : 5 éléments



In [ ]:
showPartitions(data3)

Partition 0 , 5 éléments
Row(code='01516X0004/S1', niveau=20.4, date='2021-12-15', time=1639540800000)
Row(code='01516X0004/S1', niveau=20.41, date='2021-12-14', time=1639440000000)
Row(code='01516X0004/S1', niveau=20.41, date='2021-12-13', time=1639353600000)
Row(code='01516X0004/S1', niveau=20.41, date='2021-12-12', time=1639270800000)
Row(code='01516X0004/S1', niveau=20.41, date='2021-12-11', time=1639180800000)

Partition 1 , 5 éléments
Row(code='01516X0004/S1', niveau=20.4, date='2021-12-10', time=1639094400000)
Row(code='01516X0004/S1', niveau=20.4, date='2021-12-09', time=1639090800000)
Row(code='01516X0004/S1', niveau=20.41, date='2021-12-08', time=1638921600000)
Row(code='01516X0004/S1', niveau=20.4, date='2021-12-07', time=1638896400000)
Row(code='01516X0004/S1', niveau=20.42, date='2021-12-06', time=1638806400000)

Partition 2 , 5 éléments
Row(code='01516X0004/S1', niveau=20.42, date='2021-12-05', time=1638676800000)
Row(code='01516X0004/S1', niveau=20.42, date='2021-12-04',

## Question 3 : Fonction niveauMoyen

En vous inspirant de la fonctions showPartitions définir une fonction qui détermine le niveau moyen dans chaque partition. Le résultat est (numP, code, niveauMoyen) avec numP étant le numéro de partition.

In [ ]:
# exemple d'invocation de la fonction partsize définie ci-dessus pour définir un dataframe.
pointPartSize = point.rdd.mapPartitionsWithIndex(partSize).toDF(["numP", "size"])
display(pointPartSize)

,numP,size
0,0,5
1,1,4
2,2,4
3,3,5


In [ ]:
def getNiveauMoyen(df):

  def AVG(partID, iterateur):
    c=0
    moy=0
    suivant = next(iterateur, None)
    code = suivant.code
    while suivant is not None:
      c+=1
      moy+=suivant.niveau
      suivant = next(iterateur, None)
    return [(partID, code, moy/c)]  

  t = df.rdd.mapPartitionsWithIndex(AVG).toDF(["numP", "code","niveauMoyen"])
  return t

In [ ]:
t = getNiveauMoyen(data3)
display(t)

,numP,code,niveauMoyen
0,0,01516X0004/S1,20.408
1,1,01516X0004/S1,20.406
2,2,01516X0004/S1,20.424
3,3,01516X0004/S1,20.418


### Question 4 : NiveauTotalJour

Definir MesureParJour qui est un partitionnement de Mesure par date. 
En vous inspirant de la fonctions showPartitions,
définir une fonction qui détermine le niveau total par jour dans chaque partition. Le résultat est (numP, date, niveauTotal) 

In [ ]:
def getNiveauTotalJour(df):

  def SUM(partID, iterateur):
    
    d = {}
    suivant = next(iterateur, None)
    while suivant is not None:
      d[suivant.date] = d.get(suivant.date, 0)+ suivant.niveau
      suivant = next(iterateur, None)
    return [(partID, d)]  

  t = df.rdd.mapPartitionsWithIndex(SUM)

  for (partID, d) in t.collect():
    print("Partition", partID)
    for row in d.items():
        print(row)
    print()

  return t

In [ ]:
t = getNiveauTotalJour(data3)
display(t)

Partition 0
('2021-12-15', 20.4)
('2021-12-14', 20.41)
('2021-12-13', 20.41)
('2021-12-12', 20.41)
('2021-12-11', 20.41)

Partition 1
('2021-12-10', 20.4)
('2021-12-09', 20.4)
('2021-12-08', 20.41)
('2021-12-07', 20.4)
('2021-12-06', 20.42)

Partition 2
('2021-12-05', 20.42)
('2021-12-04', 20.42)
('2021-12-03', 20.43)
('2021-12-02', 20.43)
('2021-12-01', 20.42)

Partition 3
('2021-11-30', 20.44)
('2021-11-29', 20.43)
('2021-11-28', 20.42)
('2021-11-27', 20.4)
('2021-11-26', 20.4)



AttributeError: ignored